# ZooKeeper: 互联网无等待协调的扩展性系统

## 概要
在本文中，我们描述了ZooKeeper，一种用于协调分布式应用程序进程的服务。由于ZooKeeper是关键基础架构的一部分，因此ZooKeeper旨在提供一个简单而高性能的内核，用于在客户端构建更复杂的协调原语。它在复制的集中式服务中集成了组消息传递，共享寄存器和分布式锁定服务中的元素。 ZooKeeper公开的接口具有共享寄存器的无等待方面，其事件驱动机制类似于分布式文件系统的高速缓存失效，以提供简单而强大的协调服务。
ZooKeeper接口支持高性能服务实现。除了等待免费属性之外，ZooKeeper还为每个客户端提供FIFO请求执行保证，并为所有更改ZooKeeper状态的请求提供线性化。这些设计决策可以实现高性能处理流水线，本地服务器满足读取请求。我们针对目标工作负载显示2：1到100：1的读写比率，ZooKeeper每秒可以处理数十到数十万个事务。此性能允许ZooKeeper广泛用于客户端应用程序。

### 介绍

大规模分布式应用程序需要不同形式的协调。配置是最基本的协调形式之一。在最简单的形式中，配置只是系统过程的操作参数列表，而更复杂的系统具有动态配置参数。集团成员和领导者选举在分布式系统中也很常见：通常，流程需要知道哪些其他流程存在以及这些流程由哪些流程负责。锁构成了一个强大的协调原语，可以实现对关键资源的互斥访问。


协调的一种方法是为每种不同的协调需求开发服务。例如，Amazon Simple Queue Service专门针对排队。其他服务专门针对领导人选举和配置而制定。实现更强大的原型的服务可用于实现功能较弱的原型。例如，Chubby 是一种具有强同步保证的锁定服务。然后可以使用锁来实现领导者选举，组成员资格等。


在设计我们的协调服务时，我们不再在服务器端实现特定的原语，而是选择公开API，使应用程序开发人员能够实现自己的原语。这样的选择导致了协调内核的实现，该内核支持新原语而无需更改服务核心。这种方法可以实现适应应用程序要求的多种形式的协调，而不是将开发人员限制在一组固定的原语中。


在设计ZooKeeper的API时，我们远离了阻塞原语，例如锁。阻塞协调服务的原语可能会导致客户端速度缓慢或故障，从而导致客户端性能下降。如果处理请求取决于响应和其他客户端的故障检测，则服务本身的实现变得更加复杂。因此，我们的系统Zookeeper实现了一个API，它可以操作像文件系统那样按层次结构组织的简单无等待数据对象。实际上，ZooKeeper API类似于任何其他文件系统之一，仅查看API签名，ZooKeeper似乎是没有锁定方法，打开和关闭的Chubby。但是，实现无等待数据对象会显着区分ZooKeeper与基于阻塞原语（如锁）的系统。


虽然等待性属性对性能和容错很重要，但它不足以进行协调。我们还必须为运营提供订单保证。特别是，我们发现保证所有操作的FIFO客户端排序和可线性写入都能够有效地实现服务，并且足以实现我们的应用程序感兴趣的协调原语。事实上，我们可以使用我们的API实现对任意数量进程的共识，并且根据Herlihy的层次结构，ZooKeeper实现了一个通用对象。


ZooKeeper服务包含一组服务器，这些服务器使用复制来实现高可用性和性能。其高性能使包含大量流程的应用程序能够使用此类协调内核来管理协调的各个方面。我们能够使用简单的流水线架构实现ZooKeeper，这样我们就可以获得数百或数千个未完成的请求，同时仍能实现低延迟。这样的管道自然地使得能够以FIFO顺序从单个客户端执行操作。保证FIFO客户端顺序使客户端能够异步提交操作。通过异步操作，客户端一次可以拥有多个未完成的操作。例如，当新客户成为领导者并且必须处理元数据并相应地更新它时，此功能是可取的。如果没有多次未完成操作的可能性，初始化时间可以是秒而不是秒。


为了保证更新操作满足线性化，我们实现了一种基于领导的原子广播协议，称为Zab。然而，ZooKeeper应用程序的典型工作负载由读取操作支配，并且希望扩展读取吞吐量。在ZooKeeper中，服务器在本地处理读操作，我们不使用Zab对它们进行完全排序。


在客户端缓存数据是提高读取性能的重要技术。例如，对于进程缓存当前领导者的标识符而不是每次需要知道领导者时探测ZooKeeper是有用的。 ZooKeeper使用监视机制使客户端能够缓存数据，而无需直接管理客户端缓存。使用此机制，客户端可以监视给定数据对象的更新，并在更新时接收通知。 Chubby直接管理客户端缓存。它阻止更新以验证缓存正在更改的数据的所有客户端的缓存。在此设计下，如果这些客户端中的任何一个缓慢或出现故障，则更新会延迟。 Chubby使用租约来防止错误的客户端无限制地阻塞系统。然而，租约只能限制缓慢或有缺陷的客户的影响，而ZooKeeper监控完全避免了这个问题。


在本文中，我们将讨论ZooKeeper的设计和实现。使用ZooKeeper，我们能够实现我们的应用程序所需的所有协调原语，即使只有写入是可线性化的。为了验证我们的方法，我们展示了如何使用ZooKeeper实现一些协调原语。


总而言之，在本文中，我们的主要贡献是：        

**协调内核:** 我们提出了一种无等待协调服务，具有放松的一致性保证，可用于分布式系统。特别是，我们描述了协调内核的设计和实现，我们在许多关键应用中使用它来实现各种协调技术。           

**协调配方:** 我们展示ZooKeeper如何用于构建更高级别的协调原语，甚至是阻塞和强一致的原语，这些原语通常用于分布式应用程序。

**协调经验:** 我们分享了一些使用ZooKeeper并评估其性能的方法。

### 2 zookeeper 服务

客户端使用ZooKeeper客户端库通过客户端API向ZooKeeper提交请求。除了通过客户端API扩展ZooKeeper服务接口之外，客户端库还管理客户端和ZooKeeper服务器之间的网络连接。
在本节中，我们首先提供ZooKeeper服务的高级视图。然后，我们讨论客户端用于与ZooKeeper交互的API。

**术语**. 在本文中，我们使用客户端来表示ZooKeeper服务的用户，服务器用于表示提供ZooKeeper服务的进程，znode用于表示ZooKeeper数据中的内存数据节点，该数据节点组织在一个分层名称空间中。作为数据树。我们还使用术语update和write来指代修改数据树状态的任何操作。客户端在连接到ZooKeeper时会建立会话，并获取会话句柄，通过它们发出请求。


#### 2.1 服务概述

ZooKeeper为其客户端提供了一组数据节点（znode）的抽象，这些节点根据分层名称空间进行组织。此层次结构中的znode是客户端通过ZooKeeper API操作的数据对象。分层名称空间通常用于文件系统。它是组织数据对象的理想方式，因为用户习惯于这种抽象，它可以更好地组织应用程序元数据。为了引用给定的znode，我们对文件系统路径使用标准的UNIX表示法。例如，我们使用/ A / B / C来表示znode C的路径，其中C以B为父，B以A为父。所有znode都存储数据，除了短暂的znode之外的所有znode都可以有子节点。

![1.png](images/1.png)

客户端可以创建两种类型的znode：

常规的：客户端通过创建来操作常规znode并明确删除它们;

短暂的：客户端创建这样的znode，他们要么显式删除它们，要么让系统在创建它们的会话终止时（故意或由于失败）自动删除它们。

此外，在创建新的znode时，客户端可以设置顺序标志。使用顺序标志设置创建的节点具有附加到其名称的单调递增计数器的值。如果n是新的znode而p是父znode，则n的序列值永远不会小于在p下创建的任何其他顺序znode的名称中的值。

ZooKeeper实现了监控，允许客户及时接收变更通知，而无需轮询。当客户端发出设置了监视标志的读取操作时，操作将正常完成，除非服务器承诺在返回的信息发生更改时通知客户端。'监控'是与会话相关的一次性触发器;一旦触发或会话结束，它们就会被注销。'监控'表示发生了变化，但未提供变更。例如，如果客户端在“/ foo”更改两次之前发出getData（''/ foo''，true），则客户端将获得一个监视事件，告知客户端“/ foo”的数据已更改。连接丢失事件等会话事件也会发送到监视回调，以便客户知道可能会延迟监视事件。


数据模型。 ZooKeeper的数据模型本质上是一个文件系统，它具有简化的API，只有完整的数据读写，或带有分层键的键/值表。分层命名空间对于为不同应用程序的命名空间分配子树以及设置对这些子树的访问权限非常有用。我们还利用客户端目录的概念来构建更高级别的原语，我们将在2.4节中看到。

与文件系统中的文件不同，znode不是为通用数据存储而设计的。相反，znodes映射到客户端应用程序的抽象，通常对应于用于协调目的的元数据。为了说明，在图1中我们有两个子树，一个用于应用程序1（/ app1），另一个用于应用程序2（/ app2）。应用程序1的子树实现了一个简单的组成员协议：每个客户端进程pi在/ app1下创建一个znode p i，只要进程正在运行，它就会持续存在。


尽管znodes并非设计用于通用数据存储，但ZooKeeper确实允许客户端存储一些可用于元数据或分布式计算中配置的信息。例如，在基于领导者的应用程序中，对于刚开始了解哪个其他服务器当前是领导者的应用程序服务器非常有用。为了实现这一目标，我们可以让当前的领导者将此信息写入znode空间中的已知位置。 Znodes还具有关联的元数据，包括时间戳和版本计数器，允许客户端跟踪对znode的更改并根据znode的版本执行条件更新。


**会话**。客户端连接到ZooKeeper并启动会话。会话具有相关的超时。 Zoo-Keeper认为客户端有故障，如果它的会话超过该超时没有收到任何东西。当客户端显式关闭会话句柄或ZooKeeper检测到客户端出现故障时，会话结束。在一个会话中，客户观察到一系列反映其运营执行情况的状态变化。会话使客户端能够在ZooKeeper集合中从一个服务器透明地移动到另一个服务器，因此可以跨ZooKeeper服务器持续存在。

#### 2.2 客户端api

我们在下面给出了ZooKeeper API的相关子集，并讨论了每个请求的语义。 

**create**（path，data，flags）：创建一个znode 使用路径名路径，在其中存储data []，并返回新znode的名称。 flags允许客户端选择znode的类型：regular，ephemeral，并设置顺序标志;

**delete**（path，version）：如果znode处于预期版本，则删除znode路径; exists（path，watch）：如果是znode，则返回true
路径名路径存在，并返回false。 watch标志使客户端可以在znode上设置监视;

**getData**（path，watch）：返回数据和与znode相关联的元数据，例如版本信息。 watch标志的工作方式与exists（）的工作方式相同，只是如果znode不存在，Zoo-Keeper不设置监视;

**setData**（path，data，version）：如果版本号是znode的当前版本，则将data []写入znode路径;

**getChildren**（path，watch）：返回znode子节点的名称集;

**sync**（path）：等待操作开始时挂起的所有更新传播到客户端连接到的服务器。该路径目前被忽略。


所有方法都具有通过API提供的同步和异步版本。应用程序在需要执行单个ZooKeeper操作时使用同步API，并且没有要执行的并发任务，因此它会进行必要的ZooKeeper调用和阻塞。但是，异步API允许应用程序同时执行多个未完成的ZooKeeper操作和其他任务。 ZooKeeper客户端保证在每个操作中调用相应的回调。

请注意，ZooKeeper不使用句柄来访问znodes。相反，每个请求都包含正在操作的znode的完整路径。这种选择不仅简化了API（没有open（）或close（）方法），而且还消除了服务器需要维护的额外状态。

每种更新方法都采用预期的版本号，这样可以实现条件更新。如果znode的实际版本号与预期版本号不匹配，则更新将失败并显示意外版本错误。如果版本号为-1，则不执行版本检查。



#### 2.3 ZooKeeper 保证

ZooKeeper有两个基本的排序保证：

Linearizable写：更新ZooKeeper状态的所有请求都是可序列化的，并且尊重优先级;

FIFO：来自给定客户端的所有请求按照他们发送的顺序执行。

请注意，我们对线性化的定义是不同的
来自Herlihy [15]最初提出的那个，我们称之为A-linearizability（异步线性化）。在Herlihy的原始线性化定义中，客户端一次只能有一个未完成的操作（客户端是一个线程）。在我们的业务中，我们允许客户有多个未完成的业务，因此我们可以选择不保证同一客户的未完成业务的特定顺序或保证FIFO顺序。我们选择后者作为我们的属性。重要的是要观察到，对于可线性化对象而言，所有结果也适用于A线性化对象，因为满足A线性化的系统也满足线性化要求。因为只有更新请求是A-可线性化的，所以ZooKeeper在每个副本上本地处理读取请求。这允许服务在服务器添加到系统时线性扩展。

要了解这两个保证如何相互作用，请考虑以下方案。包括多个进程的系统选择领导者来命令工作进程。当新的领导者负责系统时，它必须更改大量配置参数，并在完成后通知其他进程。然后我们有两个重要的要求：

- 当新领导者开始进行更改时，我们不希望其他进程开始使用正在更改的配置;
- 如果新配置文件在配置完全更新之前消失，我们不希望进程使用此部分配置。

观察分布式锁定（例如Chubby提供的锁定）将有助于满足第一个要求，但对于第二个要求是不够的。使用ZooKeeper，新的领导者可以将路径指定为就绪znode;其他进程仅在该znode存在时才使用该配置。新的领导者通过删除就绪，更新各种配置znode并创建就绪来进行配置更改。所有这些更改都可以流水线化并异步发出，以快速更新配置状态。虽然更改操作的延迟大约为2毫秒，但是如果请求一个接一个地发出，则必须更新5000个不同znode的新领导者将花费10秒;通过异步发出请求，请求将花费不到一秒钟。由于顺序保证，如果进程看到就绪的znode，它还必须看到新领导者所做的所有配置更改。如果新的领导者在创建就绪znode之前死亡，则其他进程知道配置尚未最终确定并且不使用它。

上述方案仍然存在一个问题：如果进程在新的领导者开始进行更改之前看到就绪，然后在更改正在进行时开始读取配置，会发生什么。此问题通过通知的排序保证得以解决：如果客户端正在观察更改，则客户端将在更改完成后看到系统的新状态之前看到通知事件。因此，如果读取就绪znode的进程请求通知该znode的更改，它将在可以读取任何新配置之前看到通知客户端更改的通知

除了ZooKeeper之外，当客户拥有自己的通信渠道时，可能会出现另一个问题。例如，考虑两个在ZooKeeper中具有共享配置并通过共享通信通道进行通信的客户端A和B.如果A更改ZooKeeper中的共享配置并通过共享通信通道告知B更改，则B将在重新读取配置时看到更改。如果B的ZooKeeper副本稍微落后于A，则可能看不到新配置。使用上述保证B可以通过在重新读取配置之前发出写入来确保它看到最新的信息。为了更有效地处理这种情况，Zoo-Keeper提供了同步请求：当读取后，构成慢速读取。 sync使服务器在处理读取之前应用所有挂起的写入请求，而不会产生完全写入的开销。这个原语在想法上类似于ISIS的flush原语。

ZooKeeper还具有以下两种活跃性和持久性保证：如果大多数ZooKeeper服务器处于活动状态并且可以进行通信，则可以使用该服务;如果ZooKeeper服务成功响应变更请求，只要法定数量的服务器最终能够恢复，该变更就会在任何数量的故障中持续存在.
 

#### 2.4 原语的例子

在本节中，我们将展示如何使用ZooKeeper API来实现更强大的原语。 ZooKeeper服务对这些更强大的原语一无所知，因为它们完全是在客户端使用ZooKeeper客户端API实现的。一些常见的原语（如组成员身份和配置管理）也是等待的。对于其他人，例如集合点，客户需要等待事件。即使ZooKeeper等待，我们也可以使用ZooKeeper实现高效的阻塞原语。 ZooKeeper的顺序保证允许对系统状态进行有效推理，并且监视允许有效等待。

**配置管理:**  ZooKeeper可用于在分布式应用程序中实现动态配置。在其最简单的形式配置存储在znode，zc中。进程以zc的完整路径名启动。启动进程通过在watch标志设置为true的情况下读取zc来获取其配置。如果更新了zc中的配置，则通知进程并读取新配置，再次将watch标志设置为true。

请注意，在此方案中，与大多数使用监控一样，监控用于确保流程具有
最新信息。例如，如果观察zc的进程被通知zc的更改，并且在它可以发出zc的读取之前，还有三次对zc的更改，则该进程不再接收三个通知事件。这不会影响流程的行为，因为这三个事件只会通知流程已经知道的事情：它对zc的信息是陈旧的。

**Rendezvous：** 有时在分布式系统中，最终的系统配置看起来并不总是先验清楚。例如，客户端可能希望启动主进程和多个工作进程，但启动进程由调度程序完成，因此客户端不会提前知道它可以为工作人员提供的地址和端口等信息连接到主服务器的进程。我们使用rendezvous znode，zr处理这种情况，使用Zoo-Keeper，它是客户端创建的节点。客户端将zr的完整路径名作为主进程和工作进程的启动参数传递。当主设备启动时，它会在zr中填写有关正在使用的地址和端口的信息。当工人启动时，他们读取zr并将watch设置为true。如果还没有填写zr，则工作程序会在更新zr时等待通知。如果zr是一个短暂的节点，主进程和工作进程可以监视zr被删除并在客户端结束时自行清理。

**Group Membership:** 我们利用短暂的节点来实现组成员资格。具体来说，我们使用短暂节点允许我们查看创建节点的会话状态这一事实。我们首先指定一个znode，zg来表示该组。当组的进程成员启动时，它会在zg下创建一个短暂的子znode。如果每个进程都有唯一的名称或标识符，则该名称将用作子znode的名称;否则，进程使用SEQUENTIAL标志创建znode以获取唯一的名称赋值。例如，进程可以将进程信息放入子进程的数据，进程使用的地址和端口中。

在zg下创建子znode后，进程正常启动。它不需要做任何其他事情。如果进程失败或结束，则会自动删除在zg下表示它的znode。


只需列出zg的子级，进程就可以获取组信息。如果进程想要监视组成员身份的更改，则进程可以将监视标志设置为true，并在收到更改通知时刷新组信息（始终将监视标志设置为true）。

**Simple Locks:** 虽然ZooKeeper不是锁定服务，但它可用于实现锁定。使用ZooKeeper的应用程序通常使用根据其需要定制的同步原语，例如上面显示的那些。这里我们展示如何使用ZooKeeper实现锁，以显示它可以实现各种通用同步原语。
最简单的锁实现使用“锁定文件”。锁由znode表示。要获取锁定，客户端会尝试使用EPHEMERAL标志创建指定的znode。如果创建成功，则客户端持有锁。否则，客户端可以读取znode，并设置监视标志，以便在当前领导者死亡时通知。客户端在死亡或明确删除znode时释放锁定。等待锁定的其他客户端在观察到被删除的znode后再次尝试获取锁定。
虽然这种简单的锁定协议有效，但它确实存在一些问题。首先，它受到惊群效应的影响。如果有许多客户端等待获取锁定，则即使只有一个客户端可以获取锁定，它们也会在释放时争用锁定。其次，它只实现独占锁定。以下两个原语显示了如何克服这两个问题。

**Simple Locks without Herd Effect:** 我们定义一个锁znode l来实现这样的锁。直观地，我们排队请求锁定的所有客户端，并且每个客户端按请求到达的顺序获得锁定。因此，希望获得锁定的客户执行以下操作：

    
      
      Lock
      1 n = create(l + “/lock-”, EPHEMERAL|SEQUENTIAL) 
      2 C = getChildren(l, false)
      3 if n is lowest znode in C, exit
      4 p = znode in C ordered just before n
      5 if exists(p, true) wait for watch event 6 goto 2
      Unlock
      1 delete(n)
      
在Lock的第1行中使用SEQUENTIAL标志命令客户尝试获取锁定而不是所有其他尝试。如果客户端的znode在第3行具有最低序列号，则客户端持有锁。否则，客户端等待删除具有锁定或将在此客户端的znode之前接收锁定的zn-ode。通过仅观察客户端znode之前的znode，我们通过仅在释放锁定或放弃锁定请求时唤醒一个进程来避免惊群效应。一旦客户端监视的znode消失，客户端必须检查它是否现在持有锁。 （之前的锁定请求可能已被放弃，并且有一个序列号较低的znode仍在等待或持有锁定。）

释放锁定就像删除代表锁定请求的znode n一样简单。通过在创建时使用EPHEMERAL标志，崩溃的进程将自动清除任何锁定请求或释放它们可能具有的任何锁定。
总之，这种锁定方案具有以下优点：
1. 删除znode只会导致一个客户端唤醒，因为每个znode都被其他客户端监视，所以我们没有惊群效应;
2. 没有轮询或超时;
3. 由于我们实施锁定的方式，我们可以通过浏览ZooKeeper数据来看到锁争用，中断锁和调试锁定问题的数量。

**Read/Write Locks:** 为了实现读/写锁，我们稍微更改了锁定过程，并具有单独的读锁定和写锁定过程。解锁程序与全局锁定情况相同。

    Write Lock
    1 n = create(l + “/write-”, EPHEMERAL|SEQUENTIAL) 
    2 C = getChildren(l, false)
    3 if n is lowest znode in C, exit
    4 p = znode in C ordered just before n
    5 if exists(p, true) wait for event 6 goto 2
    Read Lock
    1 n = create(l + “/read-”, EPHEMERAL|SEQUENTIAL)
    2 C = getChildren(l, false)
    3 if no write znodes lower than n in C, exit
    4 p = write znode in C ordered just before n
    5 if exists(p, true) wait for event
    6 goto 3

此锁定程序与以前略有不同锁。写锁定仅在命名方面有所不同。由于读锁可以共享，因此第3行和第4行略有不同，因为只有早期的写锁定znode才能阻止客户端获得读锁定。当有多个客户端等待读锁定时，我们会看到“惊群效应”，并且当删除序列号较低的“write-”znode时会收到通知。实际上，这是一种所需的行为，所有那些读取客户端都应该被释放，因为它们现在可能已经锁定了。

**Double Barrier：** 双重障碍使客户能够同步计算的开始和结束。当由屏障阈值定义的足够的过程加入屏障时，过程开始计算并在完成后离开屏障。我们在ZooKeeper中用znode代表一个障碍，称为b。每个进程p都注册b  - 通过在进入时创建一个znode作为b的子进程，并在它准备离开时取消注册 - 删除子进程。当b的子znode的数量超过屏障阈值时，进程可以进入屏障。当所有进程都移除了他们的子进程时，进程可能会离开障碍。我们使用监控有效地等待进入和退出条件得到满足。要进入，进程会监视b的就绪子级是否存在，该子级将由导致子级数超过屏障阈值的进程创建。要离开，进程会监视某个特定子项消失，并且只有在删除了znode后才检查退出条件。



### 3. ZooKeeper 应用

我们现在描述一些使用ZooKeeper的应用程序，并简要解释它们如何使用它。我们以粗体显示每个示例的原语。

**The Fetching Service：** 
抓取是搜索引擎的重要组成部分，雅虎！抓取数十亿的网络文档。提取服务（FS）是Yahoo!的一部分。爬虫，它目前正在生产中。从本质上讲，它具有命令页面提取过程的主进程。主人向提取者提供配置，并且提取者回写通知他们的状态和健康状况。使用ZooKeeper for FS的主要优点是从主服务器故障中恢复，尽管故障可以保证可用性，并将客户端与服务器分离，允许他们通过从ZooKeeper读取状态来将他们的请求引导到健康的服务器。因此，FS使用ZooKeeper主要用于管理配置元数据，尽管它也使用ZooKeeper来选举主人（领导者选举）。

![2.png](images/2.png)

图2：具有提取服务的一个ZK服务器的工作负载。每个点代表一秒钟的样本。
图2显示了FS在三天内使用的ZooKeeper服务器的读写流量。为了生成该图，我们计算该周期中每秒的操作次数，并且每个点对应于该秒中的操作次数。我们观察到与写入流量相比，读取流量要高得多。在速率高于每秒1,000次操作的时段期间，读取：写入比率在10：1和100：1之间变化。此工作负载中的读取操作是getData（），getChildren（）和exists（），按流行程度递增。


**Katta**

Katta [17]是一个使用ZooKeeper进行协调的分布式索引器，它是非Yahoo!的一个应用。 Katta使用分片来划分索引的工作。主服务器将分片分配给从站并跟踪进度。从站可能会失败，因此主站必须重新分配负载，因为从站来去。主服务器也可能发生故障，因此其他服务器必须准备好在发生故障时接管。 Katta使用ZooKeeper来跟踪从属服务器和主服务器（组成员资格）的状态，并处理主故障转移（领导者选举）。 Katta还使用ZooKeeper跟踪和传播分片到从属的分配（配置管理）。


**Yahoo! Message Broker:**
雅虎Message Broker（YMB）是一个分布式发布 - 订阅系统。该系统管理数千个主题，客户可以发布消息并从中接收消息。主题分布在一组服务器中以提供可扩展性。使用主备份方案复制每个主题，确保将消息复制到两台计算机，以确保可靠的消息传递。构成YMB的服务器使用无共享的分布式架构，这使得协调对于正确的操作至关重要。 YMB使用ZooKeeper来管理主题的分配（配置元数据），处理系统中的机器故障（故障检测和组成员资格）以及控制系统操作。

![3.png](images/3.png)

图3显示了YMB的部分znode数据布局。每个代理域都有一个名为节点的znode，它对构成YMB服务的每个活动服务器都有一个短暂的znode。每个YMB服务器在节点下创建一个短暂的znode，其中包含负载和状态信息，通过ZooKeeper提供组成员资格和状态信息。禁止关闭和迁移等节点由构成服务的所有服务器监控，并允许集中控制YMB。主题目录为YMB管理的每个主题都有一个子znode。这些主题znode具有子znode，指示每个主题的主服务器和备份服务器以及该主题的订户。主服务器和备份服务器znode不仅允许服务器发现负责主题的服务器，而且还管理领导者选举和服务器崩溃。

![4.png](images/4.png)

### 4 zookeeper实现

ZooKeeper通过在组成服务的每个服务器上复制ZooKeeper数据来提供高可用性。我们假设服务器因崩溃而失败，此类故障服务器可能会在以后恢复。图4显示了ZooKeeper服务的高级组件。收到请求后，服务器会准备执行（请求处理器）。如果这样的请求需要服务器之间的协调（写请求），那么他们使用协议协议（原子广播的实现），最后服务器提交对Zoo-Keeper数据库的更改，完全复制到所有服务器上。合奏。在读取请求的情况下，服务器只读取本地数据库的状态并生成对请求的响应。

复制数据库是包含整个数据树的内存数据库。默认情况下，树中的每个znode最多存储1MB数据，但此最大值是可在特定情况下更改的配置参数。对于可恢复性，我们有效地将更新日志记录到磁盘，并且在将应用程序应用于内存数据库之前强制写入磁盘介质。事实上，正如Chubby [8]，我们保留了一个重放日志（在我们的例子中是一个预写日志）的已提交操作，并生成内存数据库的定期快照。

每个ZooKeeper服务器都为客户端服务。客户端只连接一台服务器来提交请求。如前所述，读取请求是从每个服务器数据库的本地副本提供的。更改服务状态，写请求的请求由协议协议处理。

作为协议协议的一部分，写请求被转发到称为leader1的单个服务器。其余的ZooKeeper服务器（称为关注者）接收消息提议，其中包含来自领导者的状态更改并同意状态更改。



#### 4.1 Request Processor

由于消息传递层是原子的，我们保证本地副本永远不会发散，尽管在任何时候某些服务器可能应用了比其他服务器更多的事务。与客户发送的请求不同，交易是幂等的。当领导者收到写入请求时，它会计算应用写入时系统的状态，并将其转换为捕获此新状态的事务。必须计算未来状态，因为可能存在尚未应用于数据库的突出事务。例如，如果客户端执行条件setData并且请求中的版本号与正在更新的znode的未来版本号匹配，则该服务生成包含新数据，新版本号和更新时间戳的setDataTXN。如果发生错误，例如版本号不匹配或要更新的znode不存在，则会生成errorTXN。

#### 4.2 Atomic Broadcast

更新ZooKeeper状态的所有请求都将转发给leader。领导者执行请求并通过原子广播协议Zab将更改广播到ZooKeeper状态。接收客户端请求的服务器在发送相应的状态更改时响应客户端。 Zab使用默认多数仲裁来决定提案，因此Zab和ZooKeeper只能在大多数服务器正确的情况下工作（即，使用2f + 1服务器，我们可以容忍f故障）。

为了实现高吞吐量，ZooKeeper尝试保持请求处理管道满。它可能在处理管道的不同部分中有数千个请求。由于状态变化取决于先前状态变化的应用，因此Zab提供比常规原子广播更强的订单保证。更具体地说，Zab保证领导者的广播变更按照发送的顺序进行传递，之前领导者的所有变更都会在广播自己的变更之前传递给已建立的领导者。

有一些实现细节可以简化我们的实现并为我们提供出色的性能。我们使用TCP进行传输，因此网络可以保留消息顺序，这样我们就可以简化实现。我们使用Zab选择的领导者作为ZooKeeper领导者，因此创建事务的相同过程也会提出它们。我们使用日志来跟踪提案作为内存数据库的预写日志，这样我们就不必将消息写入磁盘两次。

在正常操作期间，Zab确实按顺序传送所有消息，但由于Zab不会持续记录每条消息的ID，因此Zab可能会在恢复期间重新发送消息。因为我们使用幂等交易，所以只要按顺序交付，就可以接受多次交货。实际上，ZooKeeper要求Zab至少重新传递在上一个快照开始后传递的所有消息。

#### 4.3 Replicated Database

每个副本都有一个ZooKeeper状态的内存副本。当ZooKeeper服务器从崩溃中恢复时，它需要恢复此内部状态。在运行服务器一段时间后，将所有提供的消息重播到恢复状态会花费很长时间，因此ZooKeeper使用定期快照，并且只需要从快照开始后重新传递消息。我们称Zoo-Keeper快照为模糊快照，因为我们没有锁定ZooKeeper状态来拍摄快照;相反，我们对树进行深度优先扫描，原子地读取每个zn-ode的数据和元数据并将它们写入磁盘。由于生成的模糊快照可能已应用了在生成快照期间传递的状态更改的某些子集，因此结果可能与任何时间点的ZooKeeper状态都不对应。但是，由于状态更改是幂等的，只要我们按顺序应用状态更改，我们就可以应用它们两次。

例如，假设在ZooKeeper数据树中，两个节点/ foo和/ goo分别具有值f1和g1，并且当模糊快照开始时两者都处于版本1，并且随后的状态变化流到达了形状变换类型，路径，价值，新版本：

    ⟨SetDataTXN, /foo, f2, 2⟩ 
    ⟨SetDataTXN, /goo, g2, 2⟩ 
    ⟨SetDataTXN, /foo, f3, 3⟩
处理完这些状态更改后，/ foo和/ goo分别具有版本3和版本的值f3和g2。但是，模糊快照可能已经记录了/ foo和/ goo的值分别为f3和g1以及版本3和1，这不是ZooKeeper数据树的有效状态。如果服务器崩溃并恢复此快照并且Zab重新传递状态更改，则生成的状态对应于崩溃之前的服务状态。

####  4.4 Client-Server Interactions
当服务器处理写入请求时，它还会发送并清除与该更新对应的任何监视相关的通知。服务器按顺序处理写入，不同时处理其他写入或读取。这确保了严格的通知连续。请注意，服务器在本地处理通知仅客户端连接的服务器跟踪并触发该客户端的通知。

读取请求在每个服务器本地处理。每个读取请求都使用zxid进行处理和标记，该zxid与服务器看到的最后一个事务相对应。此zxid定义了读请求的部分顺序，与写请求相关。通过本地处理读取，我们获得了出色的读取性能，因为它只是本地服务器上的内存操作，并且没有磁盘活动或协议协议可以运行。这种设计选择是实现我们的目标，即具有读取占优势的工作负载的卓越性能的关键。

使用快速读取的一个缺点是不保证读取操作的优先顺序。也就是说，即使已经提交了对同一znode的更新更新，读操作也可能返回过时值。并非所有应用程序都需要优先顺序，但对于需要它的应用程序，我们已实现同步。该原语异步执行，并在所有挂起的写入其本地副本之后由领导者排序。为了保证给定的读操作返回最新的更新值，客户端调用读操作后的同步。客户端操作的FIFO顺序保证以及全局同步保证使读取操作的结果能够反映在发出同步之前发生的任何更改。在我们的实现中，我们不需要原子地广播同步，因为我们使用基于领导的算法，并且我们只是将同步操作放在领导者和服务器之间的请求队列的末尾，从而实现同步调用。为了实现这一点，追随者必须确保领导者仍然是领导者。如果有提交的待处理事务，则服务器不会怀疑领导者。如果挂起队列为空，则领导者需要发出空事务以提交并在该事务之后对同步进行排序。这具有很好的属性，当领导者负载时，不会产生额外的广播流量。在我们的实施中，设置超时以使领导者在追随者放弃之前意识到他们不是领导者，所以我们不发布空交易。

ZooKeeper服务器按FIFO顺序处理来自客户端的请求。响应包括响应相对于的zxid。在没有活动的间隔期间，即使是心跳消息也包括客户端连接到的服务器所看到的最后一个zxid。如果客户端连接到新服务器，则该新服务器通过检查客户端的最后一个zxid与其最后一个zxid，确保其Zoo-Keeper数据的视图至少与客户端视图一样近。如果客户端具有比服务器更近的视图，则服务器不会重新建立与客户端的会话，直到服务器赶上。保证客户端能够找到具有最近系统视图的另一台服务器，因为客户端只能看到已复制到大多数ZooKeeper服务器的更改。这种行为对于保证耐用性很重要。

为了检测客户端会话失败，ZooKeeper使用超时。如果没有其他服务器在会话超时期间从客户端会话中收到任何内容，则负责人确定存在故障。如果客户端足够频繁地发送请求，则无需发送任何其他消息。否则，客户端在低活动期间发送心跳消息。如果客户端无法与服务器通信以发送请求或心跳，则它将连接到不同的ZooKeeper服务器以重新建立其会话。为防止会话暂停，ZooKeeper客户端库在会话空闲s / 3 ms后发送心跳，如果没有从服务器听到2s / 3 ms，则切换到新服务器，其中s是以毫秒为单位的会话超时。

